In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import differential_evolution
from sklearn.metrics import mutual_info_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
import warnings

In [2]:
# Suprimir un warning específico (por ejemplo, el warning de UserWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
##Visualizacion del dataset
df = pd.read_csv(r'rtfDataSet.csv', header=0)
df

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,label
0,-0.536000,0.292000,-0.684000,0.123000,-0.118000,0.346000,-0.308000,-0.113000,0.401000,-0.399000,...,-0.135000,-0.063000,-0.410000,-0.223000,-0.599000,-0.136000,-0.329000,-0.132000,-0.266000,0
1,-0.496000,-0.298000,-0.529000,0.073000,-0.072000,0.107000,-0.316000,-0.066000,-0.168000,-0.099000,...,0.189000,0.007000,-0.362000,-0.151000,-0.338000,-0.031000,-0.159000,-0.097000,-0.131000,0
2,0.008000,-0.031000,-0.240000,0.178000,-0.120000,0.317000,-0.481000,0.031000,-0.077000,0.063000,...,-0.035000,0.101000,-0.098000,-0.172000,-0.290000,0.033000,-0.342000,-0.321000,-0.238000,0
3,-0.188000,-0.180000,-0.062000,-0.104000,-0.136000,-0.061000,-0.216000,-0.143000,0.068000,-0.189000,...,0.146000,-0.145000,-0.527000,-0.292000,-0.663000,-0.078000,-0.194000,-0.151000,-0.268000,0
4,0.038000,0.155000,-0.203000,-0.088000,-0.084000,-0.164000,-0.145000,-0.168000,0.008000,0.143000,...,0.019000,-0.084000,0.164000,-0.165000,-0.126000,-0.112000,0.029000,-0.138000,0.148000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,0.074967,-0.127350,-0.240811,-0.037584,0.035714,-0.431190,-0.154309,0.160049,-0.172790,0.146073,...,0.280726,-0.228779,0.195978,0.319949,0.521700,0.160157,0.226597,-0.052955,-0.098770,1
392,0.664006,-0.028111,-0.179418,-0.070884,0.048793,-0.001349,-0.187850,-0.000023,-0.260852,0.111592,...,-0.016521,-0.193006,0.130795,-0.001237,0.016746,-0.015542,0.273885,0.175237,-0.039909,1
393,-0.234197,-0.058229,-0.553610,-0.097793,-0.157780,0.226956,0.416572,-0.005744,-0.068708,-0.166337,...,0.231451,-0.174647,-0.396817,-0.260318,-0.440378,-0.063138,-0.386871,-0.627228,-0.142251,1
394,0.274072,-0.105900,0.203276,-0.061067,-0.100202,-0.404380,-0.085191,-0.036268,-0.253699,0.036598,...,0.157804,-0.054686,0.473430,0.117017,0.192226,0.084721,-0.029949,0.167364,0.000677,1


In [4]:
##Separacion en X y Y (utilizando)
X=df.iloc[:,:-1]
Y=df.iloc[:, -1]

#Objeto normalizador
scaler=MinMaxScaler()
#Se normaliza
X_norm=scaler.fit_transform(X)
#Visualizacion de X normalizado
X_norm = pd.DataFrame(X_norm)
X_norm


,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.294244,0.705043,0.183047,0.610883,0.287570,0.806475,0.179009,0.499009,0.890763,0.184259,...,0.448750,0.378264,0.459459,0.300824,0.339323,0.180758,0.348815,0.324151,0.526825,0.128294
1,0.315761,0.143673,0.263651,0.559548,0.330241,0.630611,0.174344,0.522299,0.433735,0.462037,...,0.462500,0.606916,0.529530,0.333791,0.389081,0.332945,0.448341,0.433056,0.543741,0.221914
2,0.586875,0.397716,0.413937,0.667351,0.285714,0.785136,0.078134,0.570367,0.506827,0.612037,...,0.530250,0.448836,0.623624,0.515110,0.374568,0.360933,0.509005,0.315823,0.435476,0.147712
3,0.481442,0.255947,0.506500,0.377823,0.270872,0.506990,0.232653,0.484143,0.623293,0.378704,...,0.464750,0.576570,0.377377,0.220467,0.291638,0.143440,0.403791,0.410634,0.517641,0.126907
4,0.603012,0.574691,0.433177,0.394251,0.319109,0.431199,0.274052,0.471754,0.575100,0.686111,...,0.475750,0.486944,0.438438,0.695055,0.379406,0.456560,0.371564,0.553491,0.523925,0.415395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,0.622898,0.306042,0.413515,0.446012,0.430161,0.234592,0.268624,0.634316,0.429888,0.688956,...,0.588026,0.671649,0.293515,0.717018,0.714547,0.834227,0.629533,0.680075,0.565029,0.244265
392,0.939756,0.400465,0.445440,0.411823,0.442294,0.550884,0.249067,0.554994,0.359155,0.657030,...,0.465734,0.461876,0.329323,0.672249,0.492580,0.539794,0.462993,0.710368,0.675320,0.285084
393,0.456591,0.371809,0.250853,0.384196,0.250668,0.718879,0.601500,0.552159,0.513488,0.399688,...,0.632761,0.636874,0.347701,0.309878,0.313533,0.273249,0.417879,0.287078,0.287468,0.214112
394,0.730001,0.326451,0.644449,0.421902,0.304080,0.254319,0.308927,0.537033,0.364900,0.587591,...,0.592278,0.584900,0.467782,0.907576,0.574303,0.642114,0.558029,0.515728,0.671515,0.313230


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X_norm, Y, test_size=0.2, random_state=42)

In [10]:
# Definicion de funcion de filtrado en este caso, gain ratio para las hipotesis seleccionadas
def filter_function(selected_features):

    # Calculate the mutual information score for each selected feature
    scores = [mutual_info_score(X[feature], Y) for feature in selected_features]
    #print(len(scores))
    # Calculate the Gain Ratio
    total_entropy = mutual_info_score(Y, Y)
    gain_ratio = sum(scores) / total_entropy if total_entropy != 0 else 0
    return gain_ratio

In [6]:
def objective_function(selected_features):
    # Convert binary vector to feature indices
    selected_indices = np.where(selected_features)[0]

    # Calculate Gain Ratio for the selected features
    info_gain = mutual_info_classif(X_train.iloc[:, selected_indices], Y_train, discrete_features='auto')
    intrinsic_info = [entropy(X_train.iloc[:, i]) for i in selected_indices]
    
    # Calculate Gain Ratio as the fitness score
    gain_ratio = [ig / ii if ii > 0 else 0 for ig, ii in zip(info_gain, intrinsic_info)]
    
    # Sum the Gain Ratios to get the overall fitness score (maximize Gain Ratio)
    fitness = sum(gain_ratio)
    
    return -fitness  # Minimize the negative fitness (maximize Gain Ratio)

In [12]:
bounds = [(0, 1)] * np.array(X).shape[1]
len(bounds)

result = differential_evolution(objective_function, bounds, maxiter=100, popsize=10) 

In [13]:
len(result.x)

70

In [14]:
# Extract the best feature subset
best_feature_subset = np.where(result.x > 0.89)[0]

# Print the selected features
print("Selected Features:", best_feature_subset)

Selected Features: [ 2  6  8 14 21 28 30 36 41 45 48 56 58 65]


In [7]:
# Define a wrapper function
def feature_selection_wrapper(X, Y, num_sets=5):
    # Initialize a list to store selected feature subsets
    selected_feature_subsets = []

    # Perform feature selection multiple times to generate different sets
    for _ in range(num_sets):

        # Define the bounds for feature selection
        bounds = [(0, 1)] * np.array(X).shape[1]

        # Perform differential evolution feature selection
        result = differential_evolution(objective_function, bounds, maxiter=100, popsize=10, workers=4)

        # Extract the best feature subset
        #best_feature_subset = np.where(result.x > 0.70)[0]

        # Store the selected feature subset
        selected_feature_subsets.append(result)

    return selected_feature_subsets

# Example usage:
# Replace X and Y with your dataset and target variable
# num_sets is the number of feature subsets to generate
num_sets = 5
selected_feature_subsets = feature_selection_wrapper(X_train, Y_train, num_sets)

# Print the selected feature subsets
for i, subset in enumerate(selected_feature_subsets):
    print(f"Set {i + 1} - Selected Features:", subset)